In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [2]:
! nvidia-smi

Tue Feb 21 05:33:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.56       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 45%   32C    P0    N/A /  75W |    789MiB /  4096MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
actions = np.array(['Fine', 'He', 'Hello', 'Me', 'Sorry', 'We', 'You'])
label_map = {label:num for num, label in enumerate(actions)}

In [4]:
label_map

{'Fine': 0, 'He': 1, 'Hello': 2, 'Me': 3, 'Sorry': 4, 'We': 5, 'You': 6}

In [5]:
folder = 30
sequence_length = 30
DATA_PATH = os.path.join("TestData2")
print(DATA_PATH)

TestData2


In [6]:
sequences, labels = [], []
for action in actions:
  for num in range(folder):
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH, action, str(num), "{}.npy".format(frame_num)))
      window.append(res[1404:])
    sequences.append(window)
    labels.append(label_map[action])
  print(action)

Fine
He
Hello
Me
Sorry
We
You


In [7]:
np.array(sequences).shape

(210, 30, 258)

In [8]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05)
X_train.shape

(199, 30, 258)

In [9]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
tb_callback

In [10]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [12]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback], validation_data=(X_val, y_val), verbose=1)

Epoch 1/500
7/7 [==============================] - 9s 680ms/step - loss: 1.9395 - categorical_accuracy: 0.1709 - val_loss: 1.8763 - val_categorical_accuracy: 0.3636
Epoch 2/500
7/7 [==============================] - 5s 687ms/step - loss: 1.7830 - categorical_accuracy: 0.2764 - val_loss: 1.6293 - val_categorical_accuracy: 0.3636
Epoch 3/500
7/7 [==============================] - 5s 726ms/step - loss: 1.5418 - categorical_accuracy: 0.2864 - val_loss: 1.1981 - val_categorical_accuracy: 0.4545
Epoch 4/500
7/7 [==============================] - 5s 740ms/step - loss: 1.2307 - categorical_accuracy: 0.4874 - val_loss: 0.8966 - val_categorical_accuracy: 0.6364
Epoch 5/500
7/7 [==============================] - 5s 663ms/step - loss: 1.2019 - categorical_accuracy: 0.5075 - val_loss: 0.7669 - val_categorical_accuracy: 0.9091
Epoch 6/500
7/7 [==============================] - 5s 660ms/step - loss: 1.4945 - categorical_accuracy: 0.6784 - val_loss: 1.3806 - val_categorical_accuracy: 0.5455
Epoch 7/50

In [13]:
z_predict = model.predict(X_val)
print(np.sum(z_predict.argmax(axis=1) == y_val.argmax(axis=1)) / len(y_val) * 100)

1/1 [==============================] - 1s 780ms/step
100.0


In [14]:
model.save('data_model/action7.h5')